## Analisi delle pagine scaricate

In [ ]:
from bs4 import BeautifulSoup
import sys
import glob
import numpy as np
import pandas as pd

stats = []

pages = np.load(".\\tables-rotten\\pages.npy")

df = pd.DataFrame(data=pages)
df = df.drop_duplicates(subset=0, keep="last")
pages = df.values

i=0
for p in pages:
    with open(p[2], 'r' , encoding="UTF-8") as f:
        webpage = f.read()
        soup = BeautifulSoup(webpage)    
        [x.extract() for x in soup.findAll('script')]
        [x.extract() for x in soup.findAll('style')]
        countDIV = soup.findAll("div")    
        countP = soup.findAll("p")
        countLI = soup.findAll("li") 
        count = soup.findAll()
        url = p[0]
        template_type = p[1]
        #url = soup.findAll("link", {"rel" : "canonical"})# mi estraggo l'url dalla pagina; potrebbe non funzionare su tutti i siti
        #coeff = len(count)/(len(count)+len(tokens))
        coeff = len(countDIV)/(len(count))
        temp = [len(countDIV),len(countLI),len(countP) , coeff , template_type , url]
        print(temp)
        #temp = [len(countP),len(count) , url[0].get("href")]
        stats.append(temp)

        #i += 1
        #if( i%250 == 0):
            #    print(i)

In [ ]:
pages = np.load(".\\tables-rotten\\pages.npy")

df = pd.DataFrame(data=pages)
df = df.drop_duplicates(subset=0, keep="last")
pages = df.values
pages

In [ ]:
import pandas as pd
DICT = np.array([])
with open('.\\pages-rotten\\0-2.html', 'r' , encoding="UTF-8") as f:
    webpage = f.read()
    soup = BeautifulSoup(webpage)      
    tags_occurr = []
    for tag in soup.findAll():
        tags_occurr.append(tag.name)        
    tags = pd.DataFrame(data=tags_occurr)
    tags = tags.groupby([0])
    grouped_tags = tags.size()
    grouped_tags
    norm = grouped_tags.div(grouped_tags.sum(axis=0), axis=0)
    test = pd.DataFrame(columns = ["url" , "template"])
    test.loc[0] = np.zeros(2)
    for i in norm.index:
        if i in test.columns:
            test.loc[0][i] = norm[i]
        else:
            test.insert(2, i, 0)
            test.loc[0 , i] = norm[i]


In [ ]:
test

In [ ]:
import numpy as np

npstats = np.array(stats)
np.save("rotten2000test.npy" , npstats)
#npstats = np.load("test0-6000.npy")


## Conta delle occorrenze nella quantità dei DIV

In [ ]:
occurrencies = np.zeros(2000)
maxm = 0
for row in npstats :
    #if(int(row[0]) > maxm):
        #maxm = int(row[0])
    occurrencies[int(row[0])] += 1 


In [ ]:
import matplotlib.pyplot as plt

#plt.hist(results_2D[:100,:])  # arguments are passed to np.histogram
plt.bar(range(2000), occurrencies)
plt.title("Istogramma della conta dei DIV")
plt.show()

## Conta delle occorrenze nel rapporto tra DIV e conta totale dei tag

In [ ]:
coeff_array = np.rint(np.asarray(npstats[:, 3], dtype='float64')*1000)

In [ ]:
coeff_occurrencies = np.zeros(500)

for crow in coeff_array :
    coeff_occurrencies[int(crow)] += 1 



In [ ]:
import matplotlib.pyplot as plt

#plt.hist(results_2D[:100,:])  # arguments are passed to np.histogram
plt.bar(range(500), coeff_occurrencies)
plt.title("Istogramma delle percentuali dei tag DIV rispetto alla conta totale dei tag")
plt.show()

## Analisi risultati

In [ ]:
import pandas as pd
pages = np.load("rotten2000test.npy")
df = pd.DataFrame(data=pages)
df

In [ ]:
t1 = 0.23
t2 = 0.30

overI2 = df.loc[(df[3].astype(float) >= t2)][0].size
T2 = df.loc[ (df[4] == "2")][0].size
T2_relevant = df.loc[ (df[4] == "2") & (df[3].astype(float) >= t2)][0].size

betweenI1_I2 = df.loc[(df[3].astype(float) < t2) & (df[3].astype(float) > t1) ][0].size
T1 = df.loc[(df[4] == "1")][0].size
T1_relevant = df.loc[((df[3].astype(float) < t2) & (df[3].astype(float) > t1)) & (df[4] == "1")][0].size


underI1 = df.loc[(df[3].astype(float) <= t1)][0].size
T0 = df.loc[(df[4] == "0")][0].size
T0_relevant = df.loc[(df[3].astype(float) < t1) & (df[4] == "0")][0].size


print("Sopra I2 : "+str(overI2))
print("Pagine Template 2 : "+str(T2))
print("Rilevanti Template 2 : "+str(T2_relevant))
print("")
print("Tra I1 e I2 : "+str(betweenI1_I2))
print("Pagine Template 1' : "+str(T1))
print("Rilevanti Template 1 : "+str(T1_relevant))
print("")
print("Sotto I1 : "+str(underI1))
print("Pagine Template 0 : "+str(T0))
print("Rilevanti Template 0 : "+str(T0_relevant))
print("")
print("Precision Template 2 : "+ str(T2_relevant/overI2))
print("Recall Template 2 : "+ str(T2_relevant/T2))
print("")
print("Precision Template 1 : "+ str(T1_relevant/betweenI1_I2))
print("Recall Template 1 : "+ str(T1_relevant/T1))
print("")
print("Precision Template 0 : "+ str(T0_relevant/underI1))
print("Recall Template 0 : "+ str(T0_relevant/T0))

In [ ]:
print("Precision Schede : "+ str(int_schede/over23))
print("Recall Schede : "+ str(int_schede/schede))
print("")
print("Precision Tags : "+ str(int_tags/between21_23))
print("Recall Tags : "+ str(int_tags/tags))



## Clustering

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
X = npstats[:,:3]
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
import seaborn as sns; sns.set() 

plt.scatter(X[:, 0],, X[:, 2] X[:, 2], c=y_kmeans, s=50)

#centers = kmeans.cluster_centers_
#plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [ ]:
y_kmeans = y_kmeans.reshape((2212, 1))

In [ ]:
print(npstats.shape)
print(y_kmeans.shape)

In [ ]:
np.set_printoptions(threshold=9999)
results = np.append(npstats, y_kmeans, axis=1)

In [ ]:
results

In [ ]:
n_schede = df.loc[df[2].str.contains("/schede/")].size
n_tag = df.loc[df[2].str.contains("/tag/")].size

print(n_schede)
print(n_tag)

In [ ]:
n_ones = (df.loc[(df[3]) == "0"]).size
print(n_ones)